In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
from textattack.augmentation import EmbeddingAugmenter, EasyDataAugmenter
import numpy as np
import tqdm

In [ ]:
"""LOADING DATA"""
file = open('./data/semeval.txt', 'r')
headers = file.readline().split('\t')

file = open('./data/semeval.txt', 'r')
data = list()
for line in file:
    data.append(line.split('\t'))
data = data[1:]

data = pd.DataFrame(data, columns=headers)
data['relatedness_score'] = pd.to_numeric(data['relatedness_score'])

file = open('./data/semeval_train.txt', 'r')
headers = file.readline().split('\t')

file = open('./data/semeval_train.txt', 'r')
data_train = list()
for line in file:
    data_train.append(line.split('\t'))
data_train = data_train[1:]

data_train = pd.DataFrame(data_train, columns=headers)
data_train['relatedness_score'] = pd.to_numeric(data_train['relatedness_score'])
data_train.tail()

In [ ]:
data = data.iloc[:len(data)-1,:]

#### Aug 1

In [ ]:
sentences = []
for i in range(len(data_train)):
    for _ in range(3):
        s1 = data_train['sentence_A'][i]
        s2 = data_train['sentence_B'][i]
        score = data_train['relatedness_score'][i]

        s1 = aug.augment(s1)
        s2 = aug.augment(s2)

        sentences.append([s1, s2, score])
        
pd.DataFrame(sentences, columns=['sent_1', 'sent_2', 'sim']).to_csv('./data/semeval_train_aug.csv', index=False)

In [ ]:
sentences = []
for i in range(len(data)):
    for _ in range(3):
        s1 = data['sentence_A'][i]
        s2 = data['sentence_B'][i]
        score = data['relatedness_score'][i]

        s1 = aug.augment(s1)
        s2 = aug.augment(s2)

        sentences.append([s1, s2, score])

In [ ]:
pd.DataFrame(sentences, columns=['sent_1', 'sent_2', 'sim']).to_csv('./data/semeval_test_aug.csv', index=False)

#### Aug 2

In [ ]:
embed_aug = EmbeddingAugmenter()
easy_aug = EasyDataAugmenter()

sentences = []
for i in range(len(data_train)):
    s1 = data_train['sentence_A'][i]
    s2 = data_train['sentence_B'][i]
    score = data_train['relatedness_score'][i]

    easy_sentences1 = easy_aug.augment(s1)
    easy_sentences2 = easy_aug.augment(s2)
    
#     embed_sentences1 = list()
#     embed_sentences2 = list()
    
#     for _ in range(3):
#         embed_sentences1.append(embed_aug.augment(s1)[0])
#         embed_sentences2.append(embed_aug.augment(s2)[0])
        
    for j in range(len(easy_sentences1)):
        sentences.append([easy_sentences1[j], easy_sentences2[j], score])
        
#     for k in range(len(embed_sentences1)):
#         sentences.append([embed_sentences1[k], embed_sentences2[k], score])

pd.DataFrame(sentences, columns=['sent_1', 'sent_2', 'sim']).to_csv('./data/semeval_train_aug.csv', index=False)

In [ ]:
embed_aug = EmbeddingAugmenter()
easy_aug = EasyDataAugmenter()

sentences = []
for i in range(len(data)):
    s1 = data['sentence_A'][i]
    s2 = data['sentence_B'][i]
    score = data['relatedness_score'][i]

    easy_sentences1 = easy_aug.augment(s1)
    easy_sentences2 = easy_aug.augment(s2)
    
#     embed_sentences1 = list()
#     embed_sentences2 = list()
    
#     for _ in range(3):
#         embed_sentences1.append(embed_aug.augment(s1)[0])
#         embed_sentences2.append(embed_aug.augment(s2)[0])
        
    for j in range(len(easy_sentences1)):
        sentences.append([easy_sentences1[j], easy_sentences2[j], score])
        
#     for k in range(len(embed_sentences1)):
#         sentences.append([embed_sentences1[k], embed_sentences2[k], score])

pd.DataFrame(sentences, columns=['sent_1', 'sent_2', 'sim']).to_csv('./data/semeval_test_aug.csv', index=False)

#### Aug 3

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
sentences = list()

for i in tqdm.tqdm(range(len(data_train))):
    sentence1 = data_train['sentence_A'][i]
    sentence2 = data_train['sentence_B'][i]
    score = data_train['relatedness_score'][i]
    
    # To KO
    ko_result1 = translator.translate(sentence1, src='en', dest='ko').text
    ko_result2 = translator.translate(sentence2, src='en', dest='ko').text
    
    # And back
    en_result1 = translator.translate(ko_result1, src='ko', dest='en').text
    en_result2 = translator.translate(ko_result2, src='ko', dest='en').text
    
    sentences.append([en_result1, en_result2, score])
    
pd.DataFrame(sentences, columns=['sent_1', 'sent_2', 'sim']).to_csv('./data/semeval_train_ko.csv', index=False)

In [ ]:
sentences = list()

for i in tqdm.tqdm(range(len(data))):
    sentence1 = data['sentence_A'][i]
    sentence2 = data['sentence_B'][i]
    score = data['relatedness_score'][i]
    
    # To KO
    ko_result1 = translator.translate(sentence1, src='en', dest='ko').text
    ko_result2 = translator.translate(sentence2, src='en', dest='ko').text
    
    # And back
    en_result1 = translator.translate(ko_result1, src='ko', dest='en').text
    en_result2 = translator.translate(ko_result2, src='ko', dest='en').text
    
    sentences.append([en_result1, en_result2, score])
    
# pd.DataFrame(sentences, columns=['sent_1', 'sent_2', 'sim']).to_csv('./data/semeval_test_ko.csv', index=False)
df.to_csv('./data/semeval_train_ko.csv', index=False)

In [ ]:
# Occasionally, the API doesn't actually translate...
sent1_idx = list()
sent2_idx = list()
for i in range(len(df)):
    if 'e' not in df['sent_1'][i].lower():
        if 'a' not in df['sent_1'][i].lower():
            if 'i' not in df['sent_1'][i].lower():
                if 'o' not in df['sent_1'][i].lower():
                    sent1_idx.append(i)
    if 'e' not in df['sent_2'][i].lower():
        if 'a' not in df['sent_2'][i].lower():
            if 'i' not in df['sent_2'][i].lower():
                 if 'o' not in df['sent_2'][i].lower():
                    sent2_idx.append(i)

In [ ]:
sent1_fix = list()
for idx in sent1_idx:
    sent1_fix.append(translator.translate(df['sent_1'][idx], src='ko', dest='en').text)
#     time.sleep(2)

In [ ]:
sent2_fix = list()
for idx in sent2_idx:
    sent2_fix.append(translator.translate(df['sent_2'][idx], src='ko', dest='en').text)
#     time.sleep(2)